In [10]:
from scraping import scraper
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import Dense, Dropout

In [21]:
data = pd.read_csv('training_data/TRAINING_DATA_COMP9.csv')

train = data.head(len(data) - 380)
test = data.tail(380)

train = train[(train['Game_home'] > 4) & (train['Game_away'] > 4) & (train['Rest'].abs() <= 40)]
test = test[test['Rest'].notna()]

Scraper = scraper(9)
_ = Scraper.fit_pred()

In [11]:
X_train = np.array(train[Scraper.features])
y_train = np.array(train[['Win', 'Draw', 'Loss']])

X_test  = np.array(test[Scraper.features])
y_test  = np.array(test[['Win', 'Draw', 'Loss']])

# Create neural network model
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = len(Scraper.features)))
model.add(Dense(16, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(3,  activation = 'softmax'))

# Compile model
model.compile(loss      =  'categorical_crossentropy',
              optimizer =  'adam',
              metrics   = ['accuracy'])

# Train model
model.fit(X_train, y_train, epochs = 50, batch_size = 32, validation_split = 0.1, verbose = 0)

2023-05-11 21:41:32.027227: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [44]:
# Predict the result of the upcoming games and append the results to the dataframe
preds = model.predict(X_test, verbose = 0)
test['PRED.'] = list(pd.Series(1 - np.argmax(preds, axis = 1)))
test = test.rename(columns = {'Date':'DATE', 'Home':'HOME', 'Away':'AWAY', 'Result': 'RES.'})
test['ACC.'] = test['RES.'] == test['PRED.']
test[['DATE', 'HOME', 'AWAY', 'RES.', 'PRED.', 'ACC.']]

,DATE,HOME,AWAY,RES.,PRED.,ACC.
16532,2021-08-21,Manchester City,Norwich City,1,1,True
16533,2021-08-21,Aston Villa,Newcastle Utd,1,1,True
16534,2021-08-21,Brighton,Watford,1,1,True
16535,2021-08-21,Crystal Palace,Brentford,0,-1,False
16536,2021-08-21,Leeds United,Everton,0,-1,False
...,...,...,...,...,...,...
16897,2022-05-22,Brighton,West Ham,1,1,True
16898,2022-05-22,Brentford,Leeds United,-1,1,False
16899,2022-05-22,Arsenal,Everton,1,1,True
16900,2022-05-22,Crystal Palace,Manchester Utd,1,1,True


In [68]:
from sklearn.decomposition import PCA
import seaborn as sns

In [108]:
k = 20
pca = PCA(n_components = k)
pcs = pca.fit_transform(train[Scraper.features])

In [109]:
listPCs = [f'PC{i+1}' for i in range(k)]
train[listPCs] = pcs
test[listPCs]  = pca.transform(test[Scraper.features])

In [110]:
X_train = np.array(train[listPCs])
y_train = np.array(train[['Win', 'Draw', 'Loss']])

X_test  = np.array(test[listPCs])
y_test  = np.array(test[['Win', 'Draw', 'Loss']])

# Create neural network model
model = Sequential()
model.add(Dense(24, activation = 'relu', input_dim = k))
model.add(Dense(12, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(3,  activation = 'softmax'))

# Compile model
model.compile(loss      =  'categorical_crossentropy',
              optimizer =  'adam',
              metrics   = ['accuracy'])

# Train model
model.fit(X_train, y_train, epochs = 50, batch_size = 32, validation_split = 0.1, verbose = 0)

In [111]:
# Predict the result of the upcoming games and append the results to the dataframe
preds = model.predict(X_test, verbose = 0)
test['PRED.'] = list(pd.Series(1 - np.argmax(preds, axis = 1)))
test = test.rename(columns = {'Date':'DATE', 'Home':'HOME', 'Away':'AWAY', 'Result': 'RES.'})
test['ACC.'] = test['RES.'] == test['PRED.']
test[['DATE', 'HOME', 'AWAY', 'RES.', 'PRED.', 'ACC.']]

,DATE,HOME,AWAY,RES.,PRED.,ACC.
16532,2021-08-21,Manchester City,Norwich City,1,1,True
16533,2021-08-21,Aston Villa,Newcastle Utd,1,1,True
16534,2021-08-21,Brighton,Watford,1,1,True
16535,2021-08-21,Crystal Palace,Brentford,0,-1,False
16536,2021-08-21,Leeds United,Everton,0,-1,False
...,...,...,...,...,...,...
16897,2022-05-22,Brighton,West Ham,1,1,True
16898,2022-05-22,Brentford,Leeds United,-1,1,False
16899,2022-05-22,Arsenal,Everton,1,1,True
16900,2022-05-22,Crystal Palace,Manchester Utd,1,1,True


In [112]:
test['ACC.'].mean()

0.5243243243243243